
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Batch normalization</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

# Batch Normalization
* ciagła zmiana rozkład wejść do neuronów w warstwach spowalnia uczenie
  * tzw. __internal covariate shift__
* BN przeciwdziała
* pozwala na użycie wyższych współczynników uczenia
* problem może być przedstawiony na rózne sposoby
  * wykrywanie określonych obiektów na obrazach kolorowych i na czrno białych
  * funkcja rozdzielająca jest w zasadzie ta sama (lub bardzo podobna)
  * prosta transfomacja z kolorowych do czarno białych (na przykład) może pomóc
  * to jest przesunięcie dystrybucji
* niech sieć będzie głęboka
  * aktywacje k-tej warstwy są uzależnione od poprzednich wag
  * jeśli będą się one mniej zmieniać, to aktywacje na k-tym poziomie będą bardziej stabilne i mniej czułe na zmiany
  * batchnorm będzie to zapewniać

# Batch normalization
1. uczenie w mini-batchach
  * mini-batche polepszają ewaluację gradientu
    * gradient dla mini-batchu jest estymacją gradientu obliczonego dla całego zbioru
  * uczenie szybsze niż przy pojedynczych przykładach, szczególnie dla GPU
2. uczenie sgd wymaga dopasowania wielu hiper-parametrów oraz dobrej inicjalizacji
3. __covariate shift__
  * sieć ma wiele warstw
  * wejścia do neuronów są zdeterminowane przez __wiele__ neuronów warstwy poprzedniej
  * rozkład danych dla warstwy wejściowej jest stały (dla danego zbioru)
  * w trakcie uczenia rozkład dla warstw kolejnych zmienia się
  * ta zmiana tym bardziej im głębsza jest sieć
  * wiele kolejnych warstw to jak złożenie funkcji, przy czym wewnątrzna __zmienia się__ w trakcie uczenia 
  * czyli covariate shift to zmiana rozkładu aktywacji spowodowana zmianą parametrów (wag) sieci w procesie uczenia
4. niech warstwa $Z$ ma wejście z warstwy $Y$: $$z=g(w^Ty+b)$$ a $g$ sigmoidalna
  * wzrost wartości bezwzględnej $|w^Ty+b|$ prowadzi $g()$ do obszarów o niskim gradiencie
  * spowoduje to ruch wielu wymiarów w kierunku saturacji
5. użycie ReLU, dobrej inicjalizacji, małych prędkości może pomóc
6. BN
  * może pozwolic na użycie nieliniowości sigmoidalnych
  * powinno przyspieszyć uczenie
    * pojedyncze epoki są wolniejsze, ale zbieżność szybsza
    * możliwe większe prędkosci uczenia
    * większa odporność na niedobrą inicjalizację
    * zmniejszy wpływ zanikającego gradientu
    * pozwala na użycie większej liczby rodzajów funkcji aktywacji
    * ułatwia projektowanie sieci
    * jest rodzajem generalizacji

## Założenia
1. dawno wskazywano, że uczenie jest szybsze gdy wejścia są ___whitened___ (LeCun)
  * mają zerowe średnie
  * jednostkowe wariancje
  * są zdekorelowane
2. dość proste do realizacji dla danych wejściowych (chociaż pracochłonne)
  * niech __wszystkie__ wejścia mają dodatnie wartości
  * wszystkie wagi neuronów w pierwszej warstwie będą się __wspólnie__ zwiększać lub zmniejszać
  * ścieżka do minimum będzie powolna 
  * kroki przetwarzania wejść
    * usunięcie średnich
    * dekorelacja (np. PCA)
    * wyrównanie kowariancji by dla różnych wejść były w przybliżeniu równe
    * byłoby fajnie dla kazdej warstwy

### Trudności
1. wyrównywanie rozkładów będzie robione równolegle z modyfikacją wag
2. niech dla zbioru przykładów $X$ 
  * algorytm centruje wejścia 
  $$\hat{y}=y-\mathbb{E}[y]\hskip2em\text{dla}\;y=x+b,\;\mathbb{E}[y]=\frac{1}{n}\sum_iy_i$$
  * SGD _ignoruje_ istnienie centrowania i poprawia bias $b$
  $$\begin{align}
  b&=b+\Delta{}b\\
  \Delta{}b&\propto -\frac{\partial L}{\partial \hat{y}}
  \end{align}$$
  * wtedy
  $$\begin{align}y&=x+(b+\Delta b)-\mathbb{E}[x+(b+\Delta b)]\\
  &=x+b-\mathbb{E}[x+b]
  \end{align}$$
  bez żadnego wpływu
  * $b$ będzie tylko rosnąć
  * centrowanie i normalizacja muszą __współpracować__ ze sobą
3. trzeba __zapewnić__ by dla dowolnych warstości parametrów (wag) model __zawsze__ generował aktywacje o żądanych rozkładach
  * to pozwoli by gradient kosztu po parametrach uwzględniał normalizację i jej wpływ na wagi 

1. pełna normalizacja jest trudna __dla wszystkich cech__
  * niech $x$ będzie wyjściem z danej warstwy, wtedy normalizacja
  $$\widehat{x}=Norm(x,X)$$
  gdzie $X$ jest całym zbiorem
  * to wymaga policzenia dla wstecznej propagacji Jakobianów
  $$\frac{\partial\,Norm(x,X)}{\partial\,x}\hskip{2em}\text{oraz}\hskip{2em}\frac{\partial\,Norm(x,X)}{\partial\,x}$$
  * potem macierzy kowariancji $Cov[x]$ oraz $Cov[x]^{-1/2}$
  * to jest co najmniej pracochłonne
2. potrzebne jest rozwiązanie alternatywne

## Batch Normalization
1. ___uproszczenie___ normalizacja dla każdej $k$-tej cechy osobno
$$\widehat{x}_k=\frac{x_k-E[x_k]}{\sqrt{Var[x_k]}}$$
  * przyspiesza nawet jeśli cechy są od siebie zależne
    * nie są zdekorelowane
2. BN zmienia postać funkcji uczonej przez warstwę przyjmującą $x$
  * normalizacja wejść sigmoidu ograniczy do obszaru liniowego funkcji aktywacji
  * aby __warstwa BN__ potrafiła wykonywać identyczność uczymy
  $$y_k=\gamma_k\widehat{x}_k+\beta_k$$
    * to pozwoli odtworzyć oryginalne aktywacje, jeśli okażą się optymalne
      * $\beta=\mathbb{E}[x]$
      * $\gamma=\sqrt{Var[x]}$
      * parametry są uczone równolegle z wagami
3. ___uproszczenie___ każdy __mini-batch__ estymuje średnią i wariancję każdej aktywacji
  * efektywniejsze niż dla całego zbioru
    * psuje efekt SGD
  * każdy mini-batch generuje estymacje średniej i wariancji dla każdej aktywacji
  * mini-batch często (zwykle) jest mniejszy niż liczba cech
    * obliczanie pełnej kowariancji wymagałoby regularyzacji
4. Obliczenia dla mini-batchu $B$ o $m$ przykładach (dla pojedynczej cechy $i$)
$$\begin{align}
\mu_B&=\frac{1}{m}\sum_i^mx_i\\
\sigma_B^2&=\frac{1}{m}\sum_i^m(x_i-\mu_B)\\
\widehat{x}_i&=\frac{x_i-\mu_B}{\sqrt{\sigma_B^2+\epsilon}}\\
y_i&=\gamma\widehat{x}_i+\beta
\end{align}$$
  * końcową transformacją BN() jest $$y_k=BN_{\gamma, \beta}(x_k)=\gamma_k\widehat{x}_k+\beta_k$$
  gdzie $y_k$ są wyjściami z warstwy BN
  * wartości $\widehat{x}_i$ są __wewnątrz__ warstwy BN, ale to ich cechy są istotne
    * $\widehat{x}_i$ mają średnią $0$ i wariancję $1$
    * wyjście $y$ staje się wejściem do liniowej sieci $\gamma\widehat{x}_i+\beta$
    * potem następują kolejne warstwy oryginalnej sieci

### Batch Normalization a SGD
1. użycie BN wymaga uwzglednienia w algorytmie uczenia
  * transformacje są proste, gradienty proste do wyliczenia
  * możliwe do uczenia
  * transformacja liniowa - sieć zachowuje swoją pojemność
2. przekształcenia BN __nie są__ potrzebne w trakcie inferencji
  * po fazie uczenia stosujemy w warstwie BN normalizację __na całym zbiorze uczącym__
  $$\widehat{x}=\frac{x-E[x]}{\sqrt{Var[x]}}$$
3. cały algorytm
  1. uczenie warstwy BN
  2. dodanie transformacji $y_k=BN_{\gamma_k,\beta_k}(x_k)$
  3. użycie $y_k$ zamiast $x_k$
  4. uczenie calej sieci
  5. w sieci inferencji chwilowo obliczone parametry $\gamma,\beta$
  6. po zakończeniu uczenia uśrednianie (moving window) parametrów po wielu mini-batchach
  $$E[x]=E_B[\mu_B]\hskip{2em}Var[x]=\frac{m}{m-1}E_B[\sigma_B^2]$$
  gdzie $\mu_B, \sigma^2_B$ to średnia i wariancja dla mini-batchu $B$
  7. w warstwie BN zastąpienie $y=BN_{\gamma,\beta}(x)$ przez
  $$y=\frac{\gamma}{\sqrt{Var[x]+\epsilon}}x+\left(\beta-\frac{\gamma\,E[x]}{Var[x]+\epsilon}\right)$$
4. w trakcie ewaluacji wyników (inferencji) wykorzystujemy normalizację
$$\hat{x}=\frac{x-\mathbb{E}[x]}{\sqrt{Var[x]+\epsilon}}$$
wykorzystując statystyki dla całej populacji zamiast dla mini-batchy
  * te znormalizowane aktywacje mają takie same wartości jak w trakcie uczenia
5. BN można wykorzystać dla różnych rodzajów sieci, także dla konwo2lucyjnych

### Efekt regularyzacji
1. średnie i wariancje są obliczane na mini-batchach i przez podejście moving window
2. są to estymacje
3. modeyfikacje z ich wykorzystaniem będą dodawać pewien niewielki szum
4. to dodaje pewien (niewielki) efekt regularyzacyjny
  * wielkość zależna od wielkości mini-batchów
  * to tylko dodatkowy efekt, nie podstawowe zadanie

### Czy BN działa?
* aby sieć była zbieżna, konieczne jest dobre modyfikowanie parametrów, inicjalizacja, wiele dodatkowych rzeczy
* jeśli dodamy warstwy BN
  * statystyki warstw są sterowane (odpowiednimi) współczynnikami $\gamma$ (wariancja) i $\beta$ (średnia)
  * algorytm może teraz kontrolować w dużym stopniu właśnie $\beta, \gamma$ w zamian za wszystkie wagi

## Wpływ BN na prędkość uczenia
* zbyt szybkie uczenie zwykle prowadzi do wybuchu lub zaniania gradientów
* normalizacja aktywacji w sieci zabezpiecza małe zmiany przed przesunięciem aktywacji do obszarów saturacji i zanikaniu uczenia
* także zabezpiecza przed złym wpływem zbyt dużych współczynników uczenia

## Gdzie umieścić BN?
1. w pracy o BN Ioffe i Szegedy napisali, że _we add the BN transform immediately before the nonlinearity, by normalizing x=Wu+b_
  * normalizujemy $x=w^Tu+b$, by z tego policzyć aktywację $g()$
  * właściwie _bias_ może być ominięty, ponieważ normalizacja go niweluje odejmując średnią
3. wiele prac pokazuje, że odwrócenie kolejności na warstwa gęsta --> nieliniowość --> BN daje bardzo często (zwykle?) nawet lepsze wyniki
2. BN można też wykorzystać w sieciach konwolucyjnych
  * trzeba zadbać, by te same cechy w różnych miejscach były normalizowane w ten sam sposób

### Zalety
1. BN pozwala na wyższe współczynniki uczenia
  * zwykle wysokie współczynniki uczenia prowadzą do wybuchu lub też zanikania gradientów
  * BN zabezpiecza sieć przed utknięciem w rejonie saturacji
2. BN regularyzuje model
  * każdy przykład jest uczony w kontekscie swojego mini-batchu
  * sieć nie zwraca deterministycznych wartości dla przykładów
  * Dropout może być usunięty albo osłabiony
    * różne są zdania
  * można zredukować regularyzację L2
3. przyspiesza uczenie
  * zbieżność jest szybsza, co nadrabia z przewagą dodatkowe obliczenia
4. potrzebne (konieczne) jest lepsze mieszanie przykładów